# FinanceDataReader 
- https://github.com/FinanceData/FinanceDataReader
- `import financedatareader as fdr`
- `bs4`, `ploty` 의존성 확인하기

## 1.1 FinanceDataReader 라이브러리 실행하기

In [6]:
import FinanceDataReader as fdr
# 삼성전자 종목코드 : 005930
# SK하이닉스 종목코드 : 000660

## 1.2 개별종목의 가격 데이터 불러오기
- `fdr.DataReader()`

In [8]:
df=fdr.DataReader('005930') # 총 6000개 데이터

df2 = fdr.DataReader("000660", start = '2018-04-05', end = '2021-05-31')
df2

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2018-04-05,82200,83200,81200,82900,3255356,0.031095
2018-04-06,80000,80600,79300,80300,5693281,-0.031363
2018-04-09,80000,81200,79300,80600,3231256,0.003736
2018-04-10,80000,80800,79200,80400,2924054,-0.002481
2018-04-11,82400,82400,81000,81200,3053453,0.009950
...,...,...,...,...,...,...
2021-05-25,121500,123000,120500,123000,3258364,0.029289
2021-05-26,124000,124500,122000,123000,3015522,0.000000
2021-05-27,124500,126000,123000,125500,4539067,0.020325


## 1.3 한국거래소 종목코드 불러오기
- `fdr.StockListing()`

In [ ]:
stocks=fdr.StockListing('KRX') # 코스피,코스닥,코넥스 전체
stocks

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,53700,2,-600,-1.10,53700,53900,53500,12193422,654813312100,320577322935000,5969782550,STK
1,000660,KR7000660001,SK하이닉스,KOSPI,,221500,2,-4000,-1.77,221000,225500,215000,5097057,1120825076000,161252523847500,728002365,STK
2,373220,KR7373220003,LG에너지솔루션,KOSPI,,358000,2,-2000,-0.56,356500,359500,353500,93449,33314297500,83772000000000,234000000,STK
3,207940,KR7207940008,삼성바이오로직스,KOSPI,,1055000,1,41000,4.04,1023000,1078000,1022000,138330,146416582000,75088570000000,71174000,STK
4,005380,KR7005380001,현대차,KOSPI,,212000,1,3500,1.68,208000,213500,205500,850551,177482931000,44396232492000,209416191,STK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2863,236030,KR7236030003,씨알푸드,KONEX,일반기업부,999,1,69,7.42,999,999,999,1,999,2031298668,2033332,KNX
2864,140660,KR7140660002,위월드,KONEX,일반기업부,792,0,0,0.00,0,0,0,0,0,1969903584,2487252,KNX
2865,288490,KR7288490006,나라소프트,KONEX,일반기업부,95,1,4,4.40,103,103,92,24939,2345638,1659799815,17471577,KNX
2866,413300,KR7413300005,티엘엔지니어링,KONEX,일반기업부,778,2,-22,-2.75,680,900,680,75,51537,1051478670,1351515,KNX


In [62]:
len(stocks)

2868

In [12]:
import time

t1 = time.time()
stocks=fdr.StockListing('KRX') 
t2 = time.time()

print(t2-t1)

8.532766819000244


In [66]:
def StockListing():
    
    import requests
    import pandas as pd
    import json

    try:
        from pandas import json_normalize
    except ImportError:
        from pandas.io.json import json_normalize
    
    data = {'bld': 'dbms/comm/finder/finder_stkisu', 'locale': 'ko_KR', }
    #요청시 넘겨줄 pramaters
    params ={
        'bld': 'dbms/MDC/STAT/standard/MDCSTAT01901',
        'locale': 'ko_KR',
        'mktId': 'ALL',
        'share': '1',
        'csvxls_isNo': 'false'
    }
    
    #요청시 헤더 정보
    headers = {
        'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201',
        'Upgrade-Insecure-Requests': '1', 
        # HTTP Content-Security-Policy (CSP) upgrade-insecure-requests 지시문은 
        # 사이트의 모든 비보안 URL(HTTP를 통해 제공되는 URL)을 
        # 보안 URL(HTTPS를 통해 제공되는)로 대체된 것처럼 처리하도록 사용자 에이전트에 지시
        # [출처] http 헤더 Upgrade-Insecure-Requests|작성자 겨울나무
        'User-Agent': 'Mozilla/5.0'
    }
    
    r = requests.post('http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd', params, headers=headers)
    print(r)
    
    jo = json.loads(r.text)
    df = json_normalize(jo, 'OutBlock_1')
    df_info = df[['ISU_SRT_CD', 'ISU_ABBRV']]
    
    df_info.columns = ['Symbol', 'Name']
    return df_info

stock_info = StockListing()

<Response [200]>


# 종목에 해당하는 종목코드 찾기

In [64]:
# Example) '바나나'에 해당되는 정보찾기
import pandas as pd

fruits = ['사과', '바나나', '딸기', '복숭아', '수박', '참외', '포도']
price = [1000, 2000, 3000, 4000, 5000, 6000, 7000]
market = ['A마트', 'B마트', 'C마트', 'D마트', 'E마트', 'F마트', 'G마트']
df = pd.DataFrame(
    {"과일":fruits, "가격":price, "파는곳":market})

df.index = range(1,8)

df

,과일,가격,파는곳
1,사과,1000,A마트
2,바나나,2000,B마트
3,딸기,3000,C마트
4,복숭아,4000,D마트
5,수박,5000,E마트
6,참외,6000,F마트
7,포도,7000,G마트


In [65]:
condition = df['과일'] == '바나나'

df[condition]['가격'].values[0]

np.int64(2000)

In [67]:
stock_name = "AJ네트웍스"

stock_info[stock_info['Name'] == stock_name]['Symbol'].values[0]

'095570'

In [68]:
stock_name = "SK하이닉스"

if stock_name in stock_info['Name'].values:
    stock_symbol = stock_info[stock_info['Name']== stock_name]['Symbol'].values[0]
else:
    print("There is no name in KRX")

df = fdr.DataReader(stock_symbol, start = '2021')
df

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2021-01-04,124500,128000,120500,126000,7995016,0.063291
2021-01-05,124500,132500,124000,130500,7180224,0.035714
2021-01-06,132500,137000,130000,131000,8525749,0.003831
2021-01-07,132500,137500,131500,134500,5981299,0.026718
2021-01-08,136000,138000,132500,138000,8713010,0.026022
...,...,...,...,...,...,...
2025-01-17,209500,217500,209000,214500,5041329,0.021429
2025-01-20,219000,219000,210500,212000,3036234,-0.011655
2025-01-21,216500,220000,214000,218000,3323212,0.028302


# StockListing()를 저장하고 불러올 수 있는 코드를 작성

In [69]:
def StockListing():
    
    import requests
    import pandas as pd
    import json

    try:
        from pandas import json_normalize
    except ImportError:
        from pandas.io.json import json_normalize

    #요청시 넘겨줄 pramaters
    params ={
        'bld': 'dbms/MDC/STAT/standard/MDCSTAT01901',
        'locale': 'ko_KR',
        'mktId': 'ALL',
        'share': '1',
        'csvxls_isNo': 'false'
    }
    
    #요청시 헤더 정보
    headers = {
        'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201',
        'Upgrade-Insecure-Requests': '1', 
        'User-Agent': 'Mozilla/5.0'
    }
    
    r = requests.post('http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd', params, headers=headers)
    print(r)
    
    jo = json.loads(r.text)
    df = json_normalize(jo, 'OutBlock_1')
    df_info = df[['ISU_SRT_CD', 'ISU_ABBRV']]
    
    df_info.columns = ['Symbol', 'Name']
    return df_info

import time

t1 = time.time()

stock_info  = StockListing()

t2 = time.time()

print(t2 - t1)

<Response [200]>
0.8591744899749756


In [70]:
import datetime

today_year = datetime.datetime.today().year
today_month = datetime.datetime.today().month
today_day = datetime.datetime.today().day
print(today_year, today_month, today_day)

2025 1 23


In [71]:
krx_df = StockListing()
krx_df

<Response [200]>


,Symbol,Name
0,098120,마이크로컨텍솔
1,009520,포스코엠텍
2,095570,AJ네트웍스
3,006840,AK홀딩스
4,282330,BGF리테일
...,...,...
2863,000545,흥국화재우
2864,000540,흥국화재
2865,003280,흥아해운
2866,037440,희림


In [72]:
# krx_df.to_csv("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day))

In [73]:
import os

os.path.isfile("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day))

True

In [74]:
if os.path.isfile("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day)):
    krx_df = pd.read_csv("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day), index_col = 0)
else:
    krx_df = StockListing()
    krx_df.to_csv("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day))


In [75]:
krx_df

,Symbol,Name
0,098120,마이크로컨텍솔
1,009520,포스코엠텍
2,095570,AJ네트웍스
3,006840,AK홀딩스
4,282330,BGF리테일
...,...,...
2863,000545,흥국화재우
2864,000540,흥국화재
2865,003280,흥아해운
2866,037440,희림


# 함수 만들기_(stockDataReader_fn)

In [76]:
def stockDataReader_fn(stock_name, start_date = None, end_date =None): 

    import FinanceDataReader as fdr
    import datetime
    import os
    import pandas as pd

    def StockListing():
        
        import requests
        import json

        try:
            from pandas import json_normalize
        except ImportError:
            from pandas.io.json import json_normalize

        #요청시 넘겨줄 pramaters
        params ={
            'bld': 'dbms/MDC/STAT/standard/MDCSTAT01901',
            'locale': 'ko_KR',
            'mktId': 'ALL',
            'share': '1',
            'csvxls_isNo': 'false'
        }
        
        #요청시 헤더 정보
        headers = {
            'Referer': 'http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201',
            'Upgrade-Insecure-Requests': '1', 
            'User-Agent': 'Mozilla/5.0'
        }
        
        r = requests.post('http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd', params, headers=headers)
        print(r)
        
        jo = json.loads(r.text)
        df = json_normalize(jo, 'OutBlock_1')
        df_info = df[['ISU_SRT_CD', 'ISU_ABBRV']]
        
        df_info.columns = ['Symbol', 'Name']
        return df_info
    #1. 저장 파일 형식에 필요한 Data 불러오기
    today_year = datetime.datetime.today().year
    today_month = datetime.datetime.today().month
    today_day = datetime.datetime.today().day
    
    # 2. KRX 종목 DataFrame 존재 여부 확인
    if os.path.isfile("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day)):
        krx_df = pd.read_csv("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day), index_col = 0)
    else:
        krx_df = StockListing()
        krx_df.to_csv("krx_df_{}_{}_{}.csv".format(today_year, today_month, today_day))
    
    # 3. 종목 코드 찾기
    if stock_name in krx_df['Name'].values:
        symbol = krx_df[krx_df['Name'] == stock_name]['Symbol'].values[0]
    else:
        print("There is no name in KRX")
        raise "Threr is no name in KRX"
    
    stock_df = fdr.DataReader(symbol, start_date, end_date)
    
    return stock_df





In [77]:
stock_name = "LG전자"
start_date = '2020'
end_date = None
df = stockDataReader_fn(stock_name = stock_name, start_date = start_date, end_date = end_date)
df

,Open,High,Low,Close,Volume,Change
Date,,,,,,
2020-01-02,71800,72100,71000,71000,284758,-0.015257
2020-01-03,71400,71900,70800,71100,369493,0.001408
2020-01-06,70300,70500,69600,69900,337026,-0.016878
2020-01-07,69800,70600,69800,70300,265311,0.005722
2020-01-08,70300,70300,67000,67500,937438,-0.039829
...,...,...,...,...,...,...
2025-01-17,85000,85300,83500,83700,390497,-0.016451
2025-01-20,84000,84400,83800,84100,236456,0.004779
2025-01-21,84300,84800,83900,84300,352784,0.002378
